In [12]:
import pandas as pd
import numpy as np
from surprise import Dataset
from surprise import Reader

In [5]:
# dataset 로드

movie_dataset = Dataset.load_builtin('ml-100k')

df = pd.DataFrame(movie_dataset.raw_ratings)
df.columns = ['user', 'item', 'rating', 'timestamp']

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /home/gmk0904/.surprise_data/ml-100k


In [22]:
# 데이터 확인
print(df.head())
print(df.info())
print(df.describe())

  user item  rating  timestamp
0  196  242     3.0  881250949
1  186  302     3.0  891717742
2   22  377     1.0  878887116
3  244   51     2.0  880606923
4  166  346     1.0  886397596
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user       100000 non-null  object 
 1   item       100000 non-null  object 
 2   rating     100000 non-null  float64
 3   timestamp  100000 non-null  object 
dtypes: float64(1), object(3)
memory usage: 3.1+ MB
None
              rating
count  100000.000000
mean        3.529860
std         1.125674
min         1.000000
25%         3.000000
50%         4.000000
75%         4.000000
max         5.000000
